<a href="https://colab.research.google.com/github/Mehrabslashsami/langchain-agent-prototype/blob/main/LangChain_Agent_Prototype.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install langchain wikipedia transformers


In [4]:
from langchain.llms import HuggingFacePipeline
from langchain.utilities import WikipediaAPIWrapper
from transformers import pipeline

hf_pipeline = pipeline("text2text-generation", model="google/flan-t5-small", max_new_tokens=128)
llm = HuggingFacePipeline(pipeline=hf_pipeline)

wiki = WikipediaAPIWrapper()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu
/tmp/ipython-input-2620202008.py:6: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=hf_pipeline)


In [6]:
def wiki_search(query: str) -> str:
    try:
        result = wiki.run(query)
        if not result.strip():
            return "No relevant info found on Wikipedia."
        return result
    except Exception as e:
        return f"Error fetching Wikipedia data: {e}"


In [7]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

summary_prompt = PromptTemplate(
    input_variables=["text"],
    template="Summarize this text concisely:\n\n{text}"
)

summary_chain = LLMChain(llm=llm, prompt=summary_prompt)

def summarize_text(text: str) -> str:
    try:
        return summary_chain.invoke({"text": text})["text"]
    except Exception as e:
        return f"Could not summarize due to error: {e}"


/tmp/ipython-input-671247679.py:9: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  summary_chain = LLMChain(llm=llm, prompt=summary_prompt)


In [8]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

summary_prompt = PromptTemplate(
    input_variables=["text"],
    template="Summarize this text concisely:\n\n{text}"
)

summary_chain = LLMChain(llm=llm, prompt=summary_prompt)

def summarize_text(text: str) -> str:
    try:
        return summary_chain.invoke({"text": text})["text"]
    except Exception as e:
        return f"Could not summarize due to error: {e}"


In [ ]:
from textwrap import wrap

def chunk_text(text, max_chars=1500):
    return wrap(text, max_chars)

query = "Explain quantum computing in simple terms."

# Step 1: Search Wikipedia
wiki_result = wiki_search(query)
print("Wikipedia Result (preview):\n", wiki_result[:500], "...\n")  # preview first 500 chars

# Step 2: If text is short, summarize directly, else chunk & summarize each
if len(wiki_result) < 1500:
    final_answer = summarize_text(wiki_result)
else:
    chunks = chunk_text(wiki_result, max_chars=1500)[:2]  # limit to first 2 chunks for speed
    summaries = []
    for i, chunk in enumerate(chunks):
        print(f"Summarizing chunk {i+1}/{len(chunks)}...")
        summaries.append(summarize_text(chunk))
    final_answer = " ".join(summaries)

print("\nFinal Answer:\n", final_answer)


Wikipedia Result (preview):
 Page: Superconducting quantum computing
Summary: Superconducting quantum computing is a branch of solid state  physics and quantum computing that implements superconducting electronic circuits using superconducting qubits as artificial atoms, or quantum dots. For superconducting qubits, the two logic states are the ground state and the excited state, denoted 
  
    
      
        
          |
        
        g
        ⟩
        
           and 
        
        
          |
        
        e ...

Summarizing chunk 1/2...


In [ ]:
!pip install gradio --quiet
import gradio as gr

def chat_agent(query):
    wiki_result = wiki_search(query)
    if len(wiki_result) < 1500:
        return summarize_text(wiki_result)
    else:
        chunks = chunk_text(wiki_result, max_chars=1500)[:2]
        summaries = [summarize_text(chunk) for chunk in chunks]
        return " ".join(summaries)

iface = gr.Interface(
    fn=chat_agent,
    inputs=gr.Textbox(lines=2, placeholder="Ask me anything..."),
    outputs="text",
    title="LangChain Agent Prototype",
    description="Ask questions, get answers from Wikipedia + summarization"
)

iface.launch()


In [ ]:
!pip install langchain-community -q